Notebook for our final project!

Team:
Nolan Jimmo
Nicole Donahue
Frederick Carlson
Xinyu Liu

In [1]:
#Imports, function def and some file reading

import numpy as np
import pandas as pd
import glob
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

def conf_matrix_to_df(conf_matrix, target_names):
    return pd.DataFrame(conf_matrix, columns=target_names, index=target_names)


#reading in EDSS Score data
EDSS_FILENAME = "data/EDSS_Scores.csv"
EDSS_scores = pd.read_csv(EDSS_FILENAME)

Find the subject ids that have valid EDSS scores to be able to just train model on these subjects data. Storing the valid subject id and scores in a dictionary with the structure: {Subject ID: (baseline score, 6mo score)}

In [2]:
valid_sids = {}
for i, row in EDSS_scores.iterrows():
    if type(row["Subject ID "]) == float:
        break
    if row["EDSS Baseline (Score out of 10) "] != np.NaN and row["EDSS 6mo (Score out of 10) "] != np.NaN:
        valid_sids[(row["Subject ID "])] = (str(row["EDSS Baseline (Score out of 10) "]), str(row["EDSS 6mo (Score out of 10) "]))
#print(valid_sids)

converting regular EDSS scores to the binary 0, or 1, for low vs. moderate/severe EDSS score. Everything up to 4 will be 0, everything 4 and above will be moderate/severe score

In [3]:
valid_sids_generalized = {}
for key, value in valid_sids.items():
    if float(value[0]) < 4:
        v1 = 0
    else:
        v1 = 1
    if float(value[1]) < 4:
        v2 = 0
    else:
        v2 = 0
    valid_sids_generalized[key] = (v1, v2)

Get filenames for the valid subject data files out of the data folder, for both the baseline and 6mo data

In [4]:
# here, it is the baseline of the gait data
gait_baseline_filenames = glob.glob("data/Processed Data - MS +/Gait/MS1 Session 1/*")
#print((gait_baseline_filenames))
removal = []
for g in gait_baseline_filenames:
    if g[-9:-4] not in valid_sids_generalized.keys():
        removal.append(g)

gait_b_filenames = [l for l in gait_baseline_filenames if l not in removal]

###NOTE: In this test below, sometimes the two lists are not the same length
# HOWEVER, the valid EDSS subject ids list is always longer, so we will always have a
# "target" for each feature set, so we should be good to go
#print(len(gait_b_filenames), len(valid_sids.keys()))


# here, it is the 6mo of the gait data
gait_6mo_filenames = glob.glob("data/Processed Data - MS +/Gait/MS1 Session 2/*")
#print((gait_baseline_filenames))
removal = []
for g in gait_baseline_filenames:
    if g[-9:-4] not in valid_sids_generalized.keys():
        removal.append(g)

gait_6_filenames = [l for l in gait_baseline_filenames if l not in removal]

# Now, loop through the valid files, get the features from each valid subject and assign
# their EDSS score as the "target"
valid_subject_data = []
cols = []
for g in gait_b_filenames:
    with open(g, 'r') as file:
        reader = csv.reader(file)
        if cols == []:
            cols = next(file).strip().split(',')
            cols.append('target')
        for row in reader:
            if row[0] != 'timestamp_start':
                row.append(valid_sids_generalized[g[-9:-4]][0])
                valid_subject_data.append(row)

# doing the exact some thing as before, just with the 6 month data
# We can just add this data straight to the valid_subject_data list because it is all going
# to be training data
# We do have to separate the for loops though because we have to add the proper EDSS value
# from the valid_sids dictionary
for g6 in gait_6mo_filenames:
    with open(g6, 'r') as file:
        reader = csv.reader(file)
        if cols == []:
            cols = next(file).strip().split(',')
            cols.append('target')
        for row in reader:
            if row[0] != 'timestamp_start':
                row.append(valid_sids_generalized[g[-9:-4]][1])
                valid_subject_data.append(row)
#print(cols)
#print(valid_subject_data[:5])

Final setup for the features dataframe and then training/testing the SVM model!

In [9]:
df = pd.DataFrame(valid_subject_data, columns=cols)
#print(df)
df.drop(df.columns[[0,1,2,9,10,13]], axis=1, inplace=True)
#print(df)

#Train the model and see what happens!
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, df.columns != 'target'], np.array(df.iloc[:, df.columns == 'target']).reshape(9486,), test_size = 0.2, random_state = 0)
svm = SVC(kernel="poly")
#print(set(y_test))
#print(x_train, y_train)
print('training')
svm.fit(x_train, y_train)
print("predicting")
svm_y_predict = svm.predict(x_test)

conf_matrix_svm = confusion_matrix(y_test, svm_y_predict)
print("\nPrinting confusion matrix")
conf_matrix_to_df(conf_matrix_svm, [0,1])
#print(conf_matrix_svm)

training
predicting

Printing confusion matrix


,0,1
0,1878,0
1,20,0
